In [7]:
%pip install pandas
import pandas as pd

# Load the CSV content into a DataFrame
#data = pd.read_csv('file/bquxjob_157d5c5d_1941367a86e.csv')
data = pd.read_csv('file/datos4.csv')

# Display the first few rows of the DataFrame to understand its structure
data.head()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: c:\Users\user\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


,Fecha,Campania,Plataforma,inversion,evento_objetivo,Instalaciones,cvr
0,2024-12-29,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,80.89,22,56,0.392857
1,2024-12-26,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,81.39,25,62,0.403226
2,2024-12-27,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,70.58,33,70,0.471429
3,2025-01-02,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAMERS)_I...,Meta,58.12,9,28,0.321429
4,2024-12-30,BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...,Meta,84.04,31,70,0.442857


In [8]:
# Convert 'Fecha' to datetime format
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Find the most recent date in the dataset
most_recent_date = data['Fecha'].max()

# Define the two periods for comparison
period_current_end = most_recent_date
period_current_start = period_current_end - pd.Timedelta(days=6)
period_previous_end = period_current_start - pd.Timedelta(days=1)
period_previous_start = period_previous_end - pd.Timedelta(days=6)

# Filter data for the two periods
df_current = data[(data['Fecha'] >= period_current_start) & (data['Fecha'] <= period_current_end)]
df_previous = data[(data['Fecha'] >= period_previous_start) & (data['Fecha'] <= period_previous_end)]

# Check the filtered data
df_current.head(), df_previous.head()

(        Fecha                                           Campania  Plataforma  \
 5  2025-01-04  BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...        Meta   
 6  2025-01-03  BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(GAMERS)_I...        Meta   
 8  2025-01-05  BOOMIT_PEIG_EC_TIKT_ADQUISI_ANDROID_(REGISTRO-...  TikTok Ads   
 11 2025-01-06  BOOMIT_PEIG_EC_TIKT_ADQUISI_ANDROID_(REGISTRO-...  TikTok Ads   
 12 2025-01-03  BOOMIT_PEIG_EC_TIKT_ADQUISI_ANDROID_(REGISTRO-...  TikTok Ads   
 
     inversion  evento_objetivo  Instalaciones       cvr  
 5       93.40               29             72  0.402778  
 6       59.91               15             30  0.500000  
 8      156.00               54            281  0.192171  
 11     164.63               67            291  0.230241  
 12     156.00               65            245  0.265306  ,
        Fecha                                           Campania  Plataforma  \
 0 2024-12-29  BOOMIT_PEIG_EC_META_ADQUISI_ANDROID_(PRIMTARJ)...        Meta 

In [9]:
# Convert 'inversion' column to numeric
data['inversion'] = pd.to_numeric(data['inversion'], errors='coerce')
df_current['inversion'] = pd.to_numeric(df_current['inversion'], errors='coerce')
df_previous['inversion'] = pd.to_numeric(df_previous['inversion'], errors='coerce')

# Function to calculate metrics for a given period and platform
def calculate_metrics(df, platform):
    df_platform = df[df['Plataforma'] == platform]
    total_inversion = df_platform['inversion'].sum()
    total_eventos = df_platform['evento_objetivo'].sum()
    costo_medio = total_inversion / total_eventos if total_eventos != 0 else 0
    return total_inversion, costo_medio

# Get unique platforms
platforms = data['Plataforma'].unique()

# Calculate metrics for each platform in both periods
results = {
}
for platform in platforms:
    inv_prev, cost_prev = calculate_metrics(df_previous, platform)
    inv_curr, cost_curr = calculate_metrics(df_current, platform)
    results[platform] = {
        'inv_prev': inv_prev,
        'cost_prev': cost_prev,
        'inv_curr': inv_curr,
        'cost_curr': cost_curr,
        'cost_change_pct': ((cost_curr - cost_prev) / cost_prev * 100) if cost_prev != 0 else 0,
        'inv_change_pct': ((inv_curr - inv_prev) / inv_prev * 100) if inv_prev != 0 else 0
    }

results

C:\Users\user\AppData\Local\Temp\ipykernel_18616\2157055148.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current['inversion'] = pd.to_numeric(df_current['inversion'], errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_18616\2157055148.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_previous['inversion'] = pd.to_numeric(df_previous['inversion'], errors='coerce')


{'Meta': {'inv_prev': np.float64(1132.44),
  'cost_prev': np.float64(5.606138613861386),
  'inv_curr': np.float64(1067.1399999999999),
  'cost_curr': np.float64(4.21794466403162),
  'cost_change_pct': np.float64(-24.7620340031801),
  'inv_change_pct': np.float64(-5.7663099148740935)},
 'TikTok Ads': {'inv_prev': np.float64(1346.12),
  'cost_prev': np.float64(2.939126637554585),
  'inv_curr': np.float64(1145.6299999999999),
  'cost_curr': np.float64(2.6955999999999998),
  'cost_change_pct': np.float64(-8.285680325676765),
  'inv_change_pct': np.float64(-14.893917332778656)},
 'Google Ads': {'inv_prev': np.float64(3204.24),
  'cost_prev': np.float64(2.2517498243148277),
  'inv_curr': np.float64(3104.0199999999995),
  'cost_curr': np.float64(2.756678507992895),
  'cost_change_pct': np.float64(22.42383581984775),
  'inv_change_pct': np.float64(-3.127730756747318)}}

In [10]:
for plt in results:
    print(f"Platform: {plt}")
    for key, value in results[plt].items():
        print(f"{key}: {value}")
    print()

Platform: Meta
inv_prev: 1132.44
cost_prev: 5.606138613861386
inv_curr: 1067.1399999999999
cost_curr: 4.21794466403162
cost_change_pct: -24.7620340031801
inv_change_pct: -5.7663099148740935

Platform: TikTok Ads
inv_prev: 1346.12
cost_prev: 2.939126637554585
inv_curr: 1145.6299999999999
cost_curr: 2.6955999999999998
cost_change_pct: -8.285680325676765
inv_change_pct: -14.893917332778656

Platform: Google Ads
inv_prev: 3204.24
cost_prev: 2.2517498243148277
inv_curr: 3104.0199999999995
cost_curr: 2.756678507992895
cost_change_pct: 22.42383581984775
inv_change_pct: -3.127730756747318

